In [1]:
from funcs import *   # library code

import os
import json
import ee
import json
import folium
import pandas as pd
import numpy as np
import math
import altair as alt
import urllib.request
from sklearn.linear_model import LinearRegression
import warnings

warnings.filterwarnings('ignore')

import subprocess
try:
    import geemap
except ImportError:
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])
    import geemap

# pip install ipygee
# import ipygee as ui

In [2]:
ee.Authenticate()

Enter verification code: 4/1AX4XfWgPK2hbiXgzlzjzjm8_NTVT_NcAKaQMLrT6mNhf6y3JcZFAr9xtHmg

Successfully saved authorization token.


In [3]:
ee.Initialize()

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
# Defines date ranges for analysis
dateRange, epochs2022 = genDates()

observedMonths = dateRange[:34]     # [2019-01, 2021-10]
formattedAxisDates = [rangeFormatter(i[0]) for i in dateRange]

# Defines regions for analysis
madagascar, madagascarCenter, subset, subsetCenter = genEEGeometries()

In [5]:
with open("config/gifParams.json") as f:
    gifParams_1, gifParams_2 = json.load(f)
f.close()

gifParams_1["region"] = madagascar
gifParams_2["region"] = madagascar

In [6]:
# creates monthly gifs of NDVI, EVI in 2019 using Sentinel 2 images

sentinel = ee.ImageCollection("COPERNICUS/S2_SR"
            ).filterBounds(madagascar
            ).filterDate('2019-01-01', '2020-01-01')
         
# compute a monthly mosaic over Madagascar using ["B2", "B4", "B8"]
monthlyMosaic = ee.List.sequence(1, 12).map(lambda x: sentinel.filter(ee.Filter.calendarRange(x, x, 'month')
                                      ).select(["B2", "B4", "B8"]
                                      ).mosaic(
                                      ).set('month', x))

sentinelGifDates = [str(i).zfill(2) + "-" + "19" for i in range(1, 13)]


# use mosaic to compute an image with [NDVI, EVI] bands
monthlyVI = ee.ImageCollection(monthlyMosaic).map(lambda x: calcVI(x))

sentinelNDVI_url = monthlyVI.select("NDVI").getVideoThumbURL(gifParams_1)
sentinelEVI_url = monthlyVI.select("EVI").getVideoThumbURL(gifParams_1)


# Download VI gif to drive
urllib.request.urlretrieve(sentinelNDVI_url, "gifs/sentinelNDVI.gif")
urllib.request.urlretrieve(sentinelEVI_url, "gifs/sentinelEVI.gif")


# Add image dates to gif
geemap.add_text_to_gif(in_gif="gifs/sentinelNDVI.gif", out_gif="gifs/sentinelNDVI.gif",
                       xy=("80%", "90%"), text_sequence=sentinelGifDates, 
                       duration=500, font_color="red")

geemap.add_text_to_gif(in_gif="gifs/sentinelEVI.gif", out_gif="gifs/sentinelEVI.gif",
                       xy=("80%", "90%"), text_sequence=sentinelGifDates,
                       duration=500, font_color="red")


# View generated gifs
geemap.show_image("gifs/sentinelNDVI.gif")
geemap.show_image("gifs/sentinelEVI.gif")

Output()

Output()

In [7]:
# creates monthly gifs of NDVI, EVI 2019-2021 using MODIS vegetation indices

# Loads MODIS VI collection
modis = ee.ImageCollection("MODIS/006/MOD13Q1"
         ).filterBounds(madagascar
         ).filterDate("2019-01-01", "2021-12-01")

modisMetadata = ee.List(modis.map(lambda x: x.metadata('system:time_start'))).getInfo()
modisGifDates = [i["properties"]["system:index"] for i in modisMetadata["features"]]


# Generate gif with EE
modisNDVI_url = modis.select("NDVI").getVideoThumbURL(gifParams_2)
modisEVI_url = modis.select("EVI").getVideoThumbURL(gifParams_2)


# Download gif to repo
urllib.request.urlretrieve(modisNDVI_url, "gifs/modisNDVI.gif")
urllib.request.urlretrieve(modisEVI_url, "gifs/modisEVI.gif")


# Add image dates to gifs
geemap.add_text_to_gif(in_gif="gifs/modisNDVI.gif", out_gif="gifs/modisNDVI.gif",
                       xy=("72.5%", "90%"), text_sequence=modisGifDates, 
                       duration=200, font_color="red")

geemap.add_text_to_gif(in_gif="gifs/modisEVI.gif", out_gif="gifs/modisEVI.gif",
                       xy=("72.5%", "90%"), text_sequence=modisGifDates, 
                       duration=200, font_color="red")


# View generated gifs
geemap.show_image("gifs/modisNDVI.gif")
geemap.show_image("gifs/modisEVI.gif")

Output()

Output()

## Cloud Masking

In [8]:
# 2019 monthly sentinel composite images with cloud masking

for i in range(len(dateRange[:12])):
    startDate, endDate = dateRange[i][0], dateRange[i][1]
    s2_sr_collection = get_s2_sr_cld_col(subset, startDate, endDate)
    s2_sr_median = s2_sr_collection.map(add_cld_shdw_mask
                                ).map(apply_cld_shdw_mask
                                ).median()

    m = folium.Map(location=subsetCenter, zoom_start=10)
    m.add_ee_layer(s2_sr_median,
                {'bands': ['B4', 'B3', 'B2'], 
                 'min': 0, 'max': 2500, 'gamma': 1.1},
                 'S2 cloud-free mosaic', True, 1, 9)

    m.add_child(folium.LayerControl())
    print(startDate + " - " + endDate)
    display(m)
    print("\n \n")

2019-01-01 - 2019-02-01



 

2019-02-01 - 2019-03-01



 

2019-03-01 - 2019-04-01



 

2019-04-01 - 2019-05-01



 

2019-05-01 - 2019-06-01


...


 

2019-08-01 - 2019-09-01



 

2019-09-01 - 2019-10-01



 

2019-10-01 - 2019-11-01



 

2019-11-01 - 2019-12-01



 

2019-12-01 - 2020-01-01


In [9]:
# layer 1: NDVI with cloud masking
# layer 2: NDVI without cloud masking
# layer 3: normalized difference between layers 1, 2


for i in range(len(dateRange[:12])):
    startDate, endDate = dateRange[i][0], dateRange[i][1]

    s2_mask, s2_noMask = get_s2_Modified(subset, startDate, endDate)  
    maskedNdvi = s2_mask.map(add_cld_shdw_mask
                     ).map(apply_cld_shdw_mask
                     ).median(
                     ).normalizedDifference(["B8", "B4"])

    unmaskedNdvi = s2_noMask.median().normalizedDifference(["B8", "B4"])

    ndviDifference = (maskedNdvi.select("nd").subtract(unmaskedNdvi.select("nd"))).divide(maskedNdvi.select("nd").add(unmaskedNdvi.select("nd")))

    m = folium.Map(location=subsetCenter, zoom_start=10)

    m.add_ee_layer(maskedNdvi,
                {"bands": ["nd"],
                 "palette": ["white", "green"]
               }, "Masked NDVI")

    m.add_ee_layer(unmaskedNdvi,
                {"bands": ["nd"],
                 "palette": ["white", "green"]
               }, "Unmasked NDVI")

    m.add_ee_layer(ndviDifference,
                {"bands": ["nd"],
                 "min": -0.25, "max": 0.25,
                 "palette": ["red", "white", "green"]
               }, "Normalized NDVI Difference")

    m.add_child(folium.LayerControl())

    print(startDate + " - " + endDate)
    display(m)
    print("\n \n")

2019-01-01 - 2019-02-01



 

2019-02-01 - 2019-03-01



 

2019-03-01 - 2019-04-01



 

2019-04-01 - 2019-05-01



 

2019-05-01 - 2019-06-01


...


 

2019-08-01 - 2019-09-01



 

2019-09-01 - 2019-10-01



 

2019-10-01 - 2019-11-01



 

2019-11-01 - 2019-12-01



 

2019-12-01 - 2020-01-01


## Feature Collections/Spatial Summaries

In [10]:
# Madagascar national parks
feats = ee.FeatureCollection([ee.Feature(ee.Geometry.Rectangle(46.942241, -24.839532, 46.702881, -24.636228),
                                         {"name": "d'Andohahela"}),
                              ee.Feature(ee.Geometry.Rectangle(47.223358, -23.723450, 46.852087, -23.550140),
                                         {"name": "Midongy Betofaka"}),
                              ee.Feature(ee.Geometry.Rectangle(44.002508, -24.390880, 43.697637, -23.974152),
                                         {"name": "Tsimanampetsotsa"}),
                              ee.Feature(ee.Geometry.Rectangle(46.753229, -16.165182, 47.087402, -16.259739),
                                         {"name": "d'Ankarafantsika"}),
                              ee.Feature(ee.Geometry.Rectangle(50.497992, -15.882093, 49.815574, -15.183482),
                                         {"name": "Masoala"}),
                              ee.Feature(ee.Geometry.Rectangle(44.554812, -20.091821, 44.767227, -19.703976),
                                         {"name": "Alan'Ankirisa"})])

In [11]:
# def ndviReducer(image, collection=feats):
#   """
#   Reduces an image over multiple ee.Geometries and returns an ee.FeatureCollection with aggregated band statistics
#   """
#   reducer = ee.Reducer.mean(
#                      ).combine(reducer2 = ee.Reducer.max(),
#                                sharedInputs = True
#                      ).combine(reducer2 = ee.Reducer.min(),
#                                sharedInputs = True)  
  
#   return image.reduceRegions(collection = feats,
#                              reducer = reducer,
#                              scale = 10,
#                              tileScale = 3).getInfo()["features"]

In [ ]:
### RUN ONCE ###

# dfLst = []
# for i in range(len(trainMonths)):
#   start, end = trainMonths[i]
#   sentinel = get_s2_sr_cld_col(madagascar, 
#                                start, end).map(add_cld_shdw_mask
#                                          ).map(apply_cld_shdw_mask
#                                          ).mosaic()
#   yearsSinceEpoch = ee.Date(start[:-2]+"15").difference(ee.Date('1970-01-01'), 'year').getInfo()  # defined at middle of month
#   ndvi = sentinel.normalizedDifference(["B8", "B4"]).rename("NDVI")
#   regionStats = ndviReducer(ndvi)
#   dfLst += list(map(lambda x: [rangeFormatter(start), yearsSinceEpoch] + list(x["properties"].values()),
#                     regionStats))
  
# df = pd.DataFrame(dfLst)
# df.columns = ["Month", "Years Since Epoch (t)", "Max NDVI", "Mean NDVI", "Min NDVI", "Region"]
# df = df.round(3)
# df
# df.to_csv("/content/drive/MyDrive/Colab Notebooks/regionalNDVI.csv", index=False)

In [12]:
df = pd.read_csv("regionalNDVI.csv")
df

,Month,Years Since Epoch (t),Max NDVI,Mean NDVI,Min NDVI,Region
0,01/19,49.039,0.977,0.764,-0.472,d'Andohahela
1,01/19,49.039,0.999,0.791,-0.869,Midongy Betofaka
2,01/19,49.039,0.905,0.503,-0.861,Tsimanampetsotsa
3,01/19,49.039,0.999,0.737,-0.386,d'Ankarafantsika
4,01/19,49.039,1.000,0.550,-0.998,Masoala
...,...,...,...,...,...,...
199,10/21,51.788,0.914,0.701,-0.984,Midongy Betofaka
200,10/21,51.788,0.934,0.196,-0.999,Tsimanampetsotsa
201,10/21,51.788,0.954,0.448,-0.248,d'Ankarafantsika
202,10/21,51.788,0.995,0.587,-0.995,Masoala


In [13]:
alt.layer(alt.Chart(df
            ).mark_line(
            ).encode(x=alt.X("Month:O", sort=df["Month"].unique()),
                     y="Mean NDVI:Q",
                     color="Region:N"), 
          alt.Chart(df
            ).mark_circle(size=45
            ).encode(x=alt.X("Month:O", sort=df["Month"].unique()),
                     y="Mean NDVI:Q",
                     color="Region:N",
                     tooltip=["Month", "Region", "Mean NDVI"])
  ).properties(title="Quarterly Mean NDVI in Madagascar",
               height=300, width=300
  ).facet(facet="Region:N", columns=3)

alt.FacetChart(...)

## Trend Analysis

In [14]:
def plotCharts(df_1, df_2, color, region):
  """
  Returns two charts where the first displays observed mean ndvi and fitted values.
  A vertical line at 11/21 seperates fitted values from predicted values.
  The second chart displays detrended mean NDVI by plotting a model's residuals.

  """
  plot_1 = alt.layer(alt.Chart(df_1).mark_line(color=color              # observed ndvi line chart
                                   ).encode(x=alt.X("Month:O",
                                                     sort=formattedAxisDates),
                                            y="Mean NDVI:Q"),
                     alt.Chart(df_1).mark_circle(size=50, color=color  # observed ndvi scatterplot
                                   ).encode(x=alt.X("Month:O",
                                                     sort=formattedAxisDates),
                                            y="Mean NDVI:Q",
                                            tooltip=["Month", "Region", "Mean NDVI"]),
                     alt.Chart(df_1).mark_line(color="red"              # fitted ndvi line chart
                                   ).encode(x=alt.X("Month:O",
                                                     sort=formattedAxisDates),
                                            y=alt.Y("Fitted Mean NDVI:Q", title="Mean NDVI")),
                     alt.Chart(df_1).mark_circle(size=50, color="red"   # fitted ndvi scatterplot
                                   ).encode(x=alt.X("Month:O",
                                                     sort=formattedAxisDates),
                                            y=alt.Y("Fitted Mean NDVI:Q", title="Mean NDVI"),
                                            tooltip=["Month", "Region", "Fitted Mean NDVI", "Residual"]),
                     alt.Chart(pd.DataFrame({'Month': ['11/21']})
                       ).mark_rule().encode(x=alt.X("Month:O",
                                                     sort=formattedAxisDates)),
                     alt.Chart(df_2).mark_line(color="red"              # predicted ndvi line chart
                                   ).encode(x=alt.X("Month:O",
                                                     sort=formattedAxisDates),
                                            y=alt.Y("Predicted Mean NDVI:Q", title="Mean NDVI")),
                     alt.Chart(df_2).mark_circle(size=50, color="red"  # predicted ndvi scatterplot
                                   ).encode(x=alt.X("Month:O",
                                                     sort=formattedAxisDates),
                                            y=alt.Y("Predicted Mean NDVI:Q", title="Mean NDVI"),
                                            tooltip=["Month", "Region", "Predicted Mean NDVI"]))
  
  plot_2 = alt.layer(alt.Chart(df_1).mark_line(color=color
                                   ).encode(x=alt.X("Month:O",
                                                     sort=df_1["Month"].unique()),
                                            y="Residual:Q"),
                     alt.Chart(df_1).mark_circle(size=50, color=color
                                   ).encode(x=alt.X("Month:O",
                                                     sort=df_1["Month"].unique()),
                                            y="Residual:Q",
                                            tooltip=["Month", "Region", "Residual"]))
  
  return alt.hconcat(plot_1.properties(title="{} Monthly Mean NDVI".format(region),
                                       height=250, width=400),
                     plot_2.properties(title="{} Detrended Monthly Mean NDVI".format(region),
                                       height=250, width=400))

In [15]:
# linear regression

charts_1, mseLinearMod = [], dict()
colors = ["#FFD13C", "#61ECB6", "#2ca02c", "#9467bd", "#17becf", "#D159F2"]

for i in list(zip(df["Region"].unique(), colors)):
  dfSubset = df[df["Region"] == i[0]]
  trainFeatures, target = dfSubset["Years Since Epoch (t)"].values.reshape(34,1), dfSubset["Mean NDVI"].values.reshape(34,1)
  predFeatures = np.reshape(epochs2022, (14, 1))

  linearMod = LinearRegression().fit(trainFeatures, target)
  dfSubset["Fitted Mean NDVI"] = np.round(linearMod.predict(trainFeatures), 3)
  dfSubset["Residual"] = np.round(target - linearMod.predict(trainFeatures), 3)
  mseLinearMod[i[0]] = np.mean(np.square(dfSubset["Residual"]))

  predDf = pd.DataFrame({"Month": formattedAxisDates[34:],
                         "Region": [i[0]] * 14,
                         "Predicted Mean NDVI": linearMod.predict(predFeatures).flatten()}).round(3)

  charts_1.append(plotCharts(df_1=dfSubset, df_2=predDf,
                             color=i[1], region=i[0]))

alt.vconcat(*charts_1)

alt.VConcatChart(...)

In [16]:
# harmonic models

charts_2, mseHarmonicMod = [], dict()

for i in list(zip(df["Region"].unique(), colors)):
    dfSubset = df[df["Region"] == i[0]]
    timeRadiansTrain = dfSubset["Years Since Epoch (t)"].values * 2 * math.pi
    timeRadiansTest = np.array(epochs2022) * 2 * math.pi
    dfSubset[["cos", "sin"]] = [[math.cos(j), math.sin(j)] for j in timeRadiansTrain]

    trainFeatures, target = dfSubset[["Years Since Epoch (t)", "cos", "sin"]].values, dfSubset["Mean NDVI"].values.reshape(34, 1)

    predFeatures = []
    for j in epochs2022:
        timeRadians = j * 2 * math.pi
        predFeatures.append([j, math.cos(timeRadians), math.sin(timeRadians)])

    harmonicMod = LinearRegression().fit(trainFeatures, target)  
    dfSubset["Fitted Mean NDVI"] = np.round(harmonicMod.predict(trainFeatures), 3)
    dfSubset["Residual"] = np.round(target - harmonicMod.predict(trainFeatures), 3)
    mseHarmonicMod[i[0]] = np.mean(np.square(dfSubset["Residual"]))

    predDf = pd.DataFrame({"Month": formattedAxisDates[34:],
                           "Region": [i[0]] * 14,
                           "Predicted Mean NDVI": harmonicMod.predict(predFeatures
                                                            ).flatten()}
              ).round(3)

    charts_2.append(plotCharts(df_1=dfSubset, df_2=predDf, 
                               color=i[1], region=i[0]))

alt.vconcat(*charts_2)

alt.VConcatChart(...)

In [17]:
# AR(2) models

charts_3, mseAR = [], dict()

for i in list(zip(df["Region"].unique(), colors)):
    dfSubset = df[df["Region"] == i[0]]

    dfSubset["Mean NDVI (t-1)"] = dfSubset["Mean NDVI"].shift(1)
    dfSubset["Mean NDVI (t-2)"] = dfSubset["Mean NDVI"].shift(2)
    dfSubset = dfSubset.rename(columns={"Mean NDVI": "Mean NDVI (t)"}  # drop first two observations with NaN's
                      ).iloc[2:,].reset_index(drop=True)

    trainFeatures = dfSubset[["Mean NDVI (t-1)", "Mean NDVI (t-2)"]].values
    target = dfSubset["Mean NDVI (t)"].values.reshape(len(dfSubset), 1)

    ar_2 = LinearRegression().fit(trainFeatures, target)  # AR(2) model

    dfSubset["Fitted Mean NDVI"] = np.round(ar_2.predict(trainFeatures), 3)
    dfSubset["Residual"] = np.round(target - ar_2.predict(trainFeatures), 3)
    mseAR[i[0]] = np.mean(np.square(dfSubset["Residual"]))

    chart = alt.layer(alt.Chart(dfSubset).mark_line(color=i[1]
                                        ).encode(x=alt.X("Month:O",
                                                          sort=formattedAxisDates),
                                                 y="Mean NDVI (t)"),
                      alt.Chart(dfSubset).mark_circle(color=i[1], size=50
                                        ).encode(x=alt.X("Month:O",
                                                          sort=formattedAxisDates),
                                                 y="Mean NDVI (t)",
                                                 tooltip=["Month", "Region", "Mean NDVI (t)"]),
                      alt.Chart(dfSubset).mark_line(color="red"
                                        ).encode(x=alt.X("Month:O",
                                                          sort=formattedAxisDates),
                                                 y="Fitted Mean NDVI"),
                      alt.Chart(dfSubset).mark_circle(color="red", size=50
                                        ).encode(x=alt.X("Month:O",
                                                          sort=formattedAxisDates),
                                                 y="Fitted Mean NDVI",
                                                 tooltip=["Month", "Region", "Fitted Mean NDVI", "Residual"]))

    charts_3.append(chart.properties(title="{} Fitted Monthly Mean NDVI".format(i[0]),
                                     height=300, width=500))

alt.vconcat(*charts_3)

alt.VConcatChart(...)

In [18]:
# MSE for training month
pd.DataFrame({"Regions": mseLinearMod.keys(),
              "Linear MSE": mseLinearMod.values(),
              "Harmonic MSE": mseHarmonicMod.values(),
              "AR(2) MSE": mseAR.values()})

,Regions,Linear MSE,Harmonic MSE,AR(2) MSE
0,d'Andohahela,0.001589,0.000385,0.000841
1,Midongy Betofaka,0.001260,0.000203,0.000659
2,Tsimanampetsotsa,0.005909,0.001512,0.001545
3,d'Ankarafantsika,0.008959,0.001233,0.002972
4,Masoala,0.004690,0.004158,0.004739
5,Alan'Ankirisa,0.019282,0.002053,0.002014


In [19]:
regionGeometries = {}

for i in feats.getInfo()["features"]:
    regionGeometries[i["properties"]["name"]] = ee.Geometry(i["geometry"])

In [20]:
# Each layer in a map represents the fitted ndvi of a national park's region that
# is calculated by fitting a robust linear regression

for i in range(len(dateRange[:12])):
    start, end = dateRange[i]
    bandNames = [['constant', 'time'], ["NDVI"]]
    m = folium.Map(location=madagascarCenter, zoom_start=5.6)

    for region, geometry in regionGeometries.items():     # returns a plot with each park as a layer
        collection = ee.ImageCollection("COPERNICUS/S2_SR"
                      ).filterBounds(geometry
                      ).filterDate(start, end
                      ).map(lambda x: x.addBands(x.metadata('system:time_start').divide(1e18).rename("t"))
                      ).map(lambda x: x.addBands(ee.Image(1).rename("constant"))
                      ).map(lambda x: x.addBands(x.normalizedDifference(["B8", "B4"]).rename("NDVI"))
                      ).map(lambda x: x.select(["constant", "t", "NDVI"]))

        # each region has its own fitted model for every month
        robustLinearMod = collection.reduce(ee.Reducer.robustLinearRegression(numX = 2, numY = 1))
        robustFittedImage = robustLinearMod.select(['coefficients']
                                          ).arrayFlatten(bandNames)

        m.add_ee_layer(robustFittedImage, {"bands": ["time_NDVI"],
                                           "palette": ["white", "green"]},
                                          "{} Fitted NDVI".format(region))

    m.add_child(folium.LayerControl())
    print(start + " - " + end)
    display(m)
    print("\n \n")

2019-01-01 - 2019-02-01



 

2019-02-01 - 2019-03-01



 

2019-03-01 - 2019-04-01



 

2019-04-01 - 2019-05-01



 

2019-05-01 - 2019-06-01


...


 

2019-08-01 - 2019-09-01



 

2019-09-01 - 2019-10-01



 

2019-10-01 - 2019-11-01



 

2019-11-01 - 2019-12-01



 

2019-12-01 - 2020-01-01


### Appendix
extra code

In [ ]:
# # 2019 monthly NDVI, EVI
# monthlyVI = monthlyMosaic.map(lambda x: calcVI(ee.Image(x)))
# for i in range(1):
#   image = ee.Image(monthlyVI.get(i))
#   m = folium.Map(location=madagascarCenter, zoom_start=5.6)
#   m.add_ee_layer(image, {"bands": ["NDVI"], 
#                          "min": -0.2, "max": 1,
#                          "palette": ["white", "green"]},
#                  "NDVI")

#   m.add_ee_layer(image, {"bands": ["EVI"], 
#                          "min": -0.2, "max": 1,
#                          "palette": ["white", "green"]},
#                  "EVI")
  
#   m.add_child(folium.LayerControl())
  
#   print("{}/19".format(i+1) + "\n")
#   display(m)
#   print("\n")

In [187]:
# # 2019 monthly sentinel composite images with cloud masking
# maskedImages = ee.List([])
# for i in range(len(months2019)):
#   startDate, endDate = months2019[i][0], months2019[i][1]
#   s2_sr_collection = get_s2_sr_cld_col(subset, startDate, endDate)
#   maskedImages = maskedImages.add(s2_sr_collection.map(add_cld_shdw_mask
#                                 ).map(apply_cld_shdw_mask
#                                 ).median())

# # gifParams_3 = {
# #   'dimensions': 1000,
# #   'framesPerSecond': 1,
# #   "region": madagascar,
# #   "min": 0, "max": 2500,
# #   "gamma": 1.1
# # }

# # maskedURL = ee.ImageCollection(maskedImages).select("B4", "B3", "B2").getVideoThumbURL(gifParams_3)
# # urllib.request.urlretrieve(maskedURL, "gifs/madagascarMaskedImages_2019.gif")

In [ ]:
# 2019 monthly composite sentinel images with cloud masking and kernel reducer

# for i in range(len(months2019)):
#   startDate, endDate = months2019[i][0], months2019[i][1]
#   s2_sr_collection = get_s2_sr_cld_col(subset, startDate, endDate)
#   demo = s2_sr_collection.map(add_cld_shdw_mask
#                         ).map(apply_cld_shdw_mask
#                         ).map(lambda x: x.reduceNeighborhood(reducer = ee.Reducer.mean(),
#                                                              kernel = ee.Kernel.circle(3))
#                         ).median()

#   m = folium.Map(location=subsetCenter, zoom_start=10)

#   m.add_ee_layer(demo,
#                 {'bands': ['B4_mean', 'B3_mean', 'B2_mean'],
#                  "min": -100, "max": 2500, "gamma": 1.1},
#                 #  'gamma': [1.1, 1.3, 0.9]},
#                  'S2 cloud-free mosaic', True, 1, 9)

#   m.add_child(folium.LayerControl())
#   print(startDate + " - " + endDate)
#   display(m)
#   print("\n \n")

In [38]:
# import subprocess

# try:
#     import geemap
# except ImportError:
#     print('Installing geemap ...')
#     subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])
# import geemap


In [39]:
# Map = geemap.Map(center=[40,-100], zoom=4)
# Map

In [ ]:
# Add Earth Engine dataset
# # Input imagery is a cloud-free Landsat 8 composite.
# l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1')

# image = ee.Algorithms.Landsat.simpleComposite(**{
#   'collection': l8.filterDate('2018-01-01', '2018-12-31'),
#   'asFloat': True
# })

# # Use these bands for prediction.
# bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10', 'B11']

# # Load training points. The numeric property 'class' stores known labels.
# points = ee.FeatureCollection('GOOGLE/EE/DEMOS/demo_landcover_labels')

# # This property of the table stores the land cover labels.
# label = 'landcover'

# # Overlay the points on the imagery to get training.
# training = image.select(bands).sampleRegions(**{
#   'collection': points,
#   'properties': [label],
#   'scale': 30
# })

# # Train a CART classifier with default parameters.
# trained = ee.Classifier.smileCart().train(training, label, bands)

# # Classify the image with the same bands used for training.
# classified = image.select(bands).classify(trained)

# # Display the inputs and the results.
# Map.centerObject(points, 11)
# Map.addLayer(image, {'bands': ['B4', 'B3', 'B2'], 'max': 0.4}, 'image')
# Map.addLayer(classified,
#              {'min': 0, 'max': 2, 'palette': ['red', 'green', 'blue']},
#              'classification')

In [40]:
# Map = geemap.Map(center=[40,-100], zoom=4)
# Map

In [27]:
# # Add Earth Engine dataset
# # This function adds a time band to the image.

# def createTimeBand(image):
#     return image.addBands(image.metadata('system:time_start').divide(1e18))

# # createTimeBand = function(image) {
# #   # Scale milliseconds by a large constant.
# #   return image.addBands(image.metadata('system:time_start').divide(1e18))
# # }

# # This function adds a constant band to the image.
# def createConstantBand(image):
#     return ee.Image(1).addBands(image)
# # createConstantBand = function(image) {
# #   return ee.Image(1).addBands(image)
# # }

# # Load the input image 'collection': projected climate data.
# collection = ee.ImageCollection('NASA/NEX-DCP30_ENSEMBLE_STATS') \
#   .filterDate(ee.Date('2006-01-01'), ee.Date('2099-01-01')) \
#   .filter(ee.Filter.eq('scenario', 'rcp85')) \
#   .map(createTimeBand) \
#   .map(createConstantBand) \
#   .select(['constant', 'system:time_start', 'pr_mean', 'tasmax_mean'])

# # Compute ordinary least squares regression coefficients.
# linearRegression = collection.reduce(
#   ee.Reducer.linearRegression(**{
#     'numX': 2,
#     'numY': 2
# }))

# # Compute robust linear regression coefficients.
# robustLinearRegression = collection.reduce(
#   ee.Reducer.robustLinearRegression(**{
#     'numX': 2,
#     'numY': 2
# }))

# # The results are array images that must be flattened for display.
# # These lists label the information along each axis of the arrays.
# bandNames = [['constant', 'time'], # 0-axis variation.
#                  ['precip', 'temp']] # 1-axis variation.

# # Flatten the array images to get multi-band images according to the labels.
# lrImage = linearRegression.select(['coefficients']).arrayFlatten(bandNames)
# rlrImage = robustLinearRegression.select(['coefficients']).arrayFlatten(bandNames)

# # Display the OLS results.
# Map.setCenter(-100.11, 40.38, 5)
# Map.addLayer(lrImage,
#   {'min': 0, 'max': [-0.9, 8e-5, 1], 'bands': ['time_precip', 'constant_precip', 'time_precip']}, 'OLS')

# # Compare the results at a specific point:
# print('OLS estimates:', lrImage.reduceRegion(**{
#   'reducer': ee.Reducer.first(),
#   'geometry': ee.Geometry.Point([-96.0, 41.0]),
#   'scale': 1000
# }).getInfo())

# print('Robust estimates:', rlrImage.reduceRegion(**{
#   'reducer': ee.Reducer.first(),
#   'geometry': ee.Geometry.Point([-96.0, 41.0]),
#   'scale': 1000
# }).getInfo())

In [28]:
# Map = geemap.Map(center=[40,-100], zoom=4)
# Map

In [29]:


# # Add Earth Engine dataset
# # Load input imagery: Landsat 7 5-year composite.
# image = ee.Image('LANDSAT/LE7_TOA_5YEAR/2008_2012')
# # print(image.getInfo())

# # Load an input region: Sierra Nevada.
# region = ee.Feature(ee.FeatureCollection('EPA/Ecoregions/2013/L3') \
#   .filter(ee.Filter.eq('us_l3name', 'Sierra Nevada')) \
#   .first())

# # Reduce the region. The region parameter is the Feature geometry.
# meanDictionary = image.reduceRegion(**{
#   'reducer': ee.Reducer.mean(),
#   'geometry': region.geometry(),
#   'scale': 30,
#   'maxPixels': 1e9
# })

# # The result is a Dictionary.  Print it.
# Map.centerObject(region, 9)
# Map.addLayer(image, {'bands': ['B4', 'B3', 'B2']}, 'Landsat-7')
# Map.addLayer(ee.Image().paint(region, 1, 3), {'palette': 'green'}, 'Mean Image')
# print(meanDictionary.getInfo())